### Exporting keys

In [29]:
import configparser
import os

In [30]:
config_file = 'keys.cfg'

In [31]:
config = configparser.ConfigParser()
config.read(config_file)

['keys.cfg']

In [118]:
os.environ['SPOTIPY_CLIENT_ID'] = config['SPOTIFY']['CLIENT_ID']
os.environ['SPOTIPY_CLIENT_SECRET'] = config['SPOTIFY']['CLIENT_SECRET']
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://ca416ba83b7c.ngrok.io'

### Spotify searching

#### Workaround for getting an OAuth 2.0 access and refresh token

* Write a simple server application that listens on a port
* Use `ngrok` to tunnel requests to that port
* Set `ngrok` URL as the redirect URI on Spotify API Dashboard
* Use the code returned to get access and refresh tokens

In [78]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

class SpotifySearch:
    def __init__(self):
        self.spotipy = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
    
    def search(self, search_query):
        """
        Search through Spotify search API and return results
        """
        return self.spotipy.search(search_query)

    def get_first_track(self,search_query):
        results = self.search(search_query)
        res = {}
        if len(results) > 0:
            try:
                first = results['tracks']['items'][0]
                res['url'] = first['external_urls']['spotify']
                res['api_url'] = first['href']
                res['uri'] = first['uri']
            except KeyError as e:
                print(f'[error] URLs/URIs not found for {search_query}')
        return res


In [77]:
spotify = SpotifySearch()
spotify.get_first_track('bts stay gold')

{'url': 'https://open.spotify.com/track/0UwQTbq7ZtBJ7v530mpaxP',
 'api_url': 'https://api.spotify.com/v1/tracks/0UwQTbq7ZtBJ7v530mpaxP',
 'uri': 'spotify:track:0UwQTbq7ZtBJ7v530mpaxP'}

In [125]:
from spotipy.oauth2 import SpotifyOAuth

scope = 'user-library-read'

creds = SpotifyOAuth(scope=scope, username='aadithpm')
creds.refresh_access_token(refresh_token='AQCrAQD7i8K7OKXc3iqhlSqL_CizKcXV85VawgHolGD9zF6KL0h3SDUVy6qawvziouzTHaRAOYF_qxjqeGD-zarXJ0HqA9tv4MAPAju-_97CXyJHfJoE-_gw9OA3UJmDlzk')

{'access_token': 'BQAzO84UpJKsO9kyNEHfiFd-rjXbOtpqQd3hKbe0rrfpMbNKyEPthSbwGDN0SYGLKR70m-FubBozW7pyzFsr62pc1huR4Z8fy-o70yuBr-1_x69XZbT54s0CsVW3tR7EUG3IM5zm0Q7oOdS0PkCf38r9te8ZWlKMN2Ufksg3JaiCUryB',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': 'user-library-read',
 'expires_at': 1596584143,
 'refresh_token': 'AQCrAQD7i8K7OKXc3iqhlSqL_CizKcXV85VawgHolGD9zF6KL0h3SDUVy6qawvziouzTHaRAOYF_qxjqeGD-zarXJ0HqA9tv4MAPAju-_97CXyJHfJoE-_gw9OA3UJmDlzk'}